In [34]:
import sys
sys.path.append('/Users/ignasivalles/Oceanography/IEO/projects/POM/pom.github.io/src')
from data_functions import *
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

file_path= "/Users/ignasivalles/Google Drive/La meva unitat/POM/data/raw/ieo2sh01a-04C6 6500 710F 02-20240923 115651.csv"

def detect_water_temp_v2(temp, time, window_size=6, only_water_val=True):
    # Calculate the rolling mean and rolling variance
    moving_avg = pd.Series(temp).rolling(window=window_size, min_periods=1).mean().to_numpy()
    moving_var = pd.Series(temp).rolling(window=window_size, min_periods=1).var().to_numpy()

    # Define a threshold for detecting similar values, using a small variance threshold
    var_threshold = np.nanmean(moving_var) * 0.1

    # Identify periods with low variance
    similar_periods = moving_var < var_threshold

    # Find the longest period of similar values
    max_length = 0
    max_start = 0
    current_start = 0
    current_length = 0

    for i in range(len(similar_periods)):
        if similar_periods[i]:
            if current_length == 0:
                current_start = i
            current_length += 1
        else:
            if current_length > max_length:
                max_length = current_length
                max_start = current_start
            current_length = 0

    if current_length > max_length:
        max_length = current_length
        max_start = current_start

    longest_similar_period = temp[max_start:max_start + max_length]
    time_similar_period = time[max_start:max_start + max_length]
    
    print(longest_similar_period)
    if only_water_val:
        return np.nanmean(longest_similar_period), time_similar_period[0]
    else:
        water_time = time[max_start:max_start + max_length]
        return longest_similar_period, water_time, moving_var, var_threshold



def get_data_from_temp_sensors(filepath, team_name='raw', lat= None, lon= None, ):

    data=extract_lat_lon_temp_time(filepath)
    time=data.time.values
    temp=data.temp.values

    water_temp, water_time = detect_water_temp_v2(temp, time)
    time_str = np.datetime_as_string(water_time, unit='h')
    fractional_time=datetime_to_decimal_year(water_time)
    
    if not lat:
      lat=data.lat.values[0]
      lon=data.lon.values[0]

    df = pd.DataFrame([{  # Approx. week 52 for Dec 24
    'Date': time_str,
    'Latitude': lat,
    'Longitude': lon,
    'Temperature': water_temp,
    'fractional_time': fractional_time,
    'Team': team_name}])

    return df

In [35]:
get_data_from_temp_sensors(file_path)

[16.8 16.8 16.8 16.7 16.9]


,Date,Latitude,Longitude,Temperature,fractional_time,Team
0,2024-09-23T09,43.46,-3.78,16.8,8.721311,raw


In [25]:
data=extract_lat_lon_temp_time(file_path)
time=data.time.values
temp=data.temp.values

detect_water_temp2(temp,time)

np.float64(17.02857142857143)